In [8]:
from datasets import load_from_disk
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
)
import torch
import random
import json


In [2]:
torch.set_num_threads(16)

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('/data/nicolasmaier/model/codet5-finetuned-seq-3/checkpoint-12000')

In [4]:
device = 'cpu'# 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)
model_gpu = model.to(device)

cpu


In [5]:
dataset = load_from_disk("/data/nicolasmaier/dataset/hf_clean_seq_dataset")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 384212
    })
    valid: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 13488
    })
    test: Dataset({
        features: ['code', 'contents', 'xmi', 'originalLine', 'input_ids', 'attention_mask', 'seq', 'labels'],
        num_rows: 22557
    })
})


In [16]:
example_orig = dataset["test"][7212]

print(tokenizer.decode(example_orig["input_ids"]))
print()
print(tokenizer.decode(example_orig["labels"]))
print()

input = tokenizer(example_orig["contents"], return_tensors="pt").to(device)

outputs = model_gpu.generate(
    input.input_ids,
    num_beams=10,
    max_length=510,
    # do_sample=True,
    temperature=0.3,
    top_k=50,
    top_p=0.8,
)
print(tokenizer.decode(outputs[0][1:]))

with open("diff_out_1.json", "w") as f:
    json.dump(json.loads(tokenizer.decode(example_orig["labels"][1:-1])), f, indent=2)
with open("diff_out_2.json", "w") as f:
    json.dump(json.loads(tokenizer.decode(outputs[0][2:-1])), f, indent=2)


<s>private int getRelativeIndex(long pos) {
    int currentSize = 0;
    currentChunkIndex = 0;

    // loop until we find the chuks holding the given position
    while (pos >= (currentSize += binaryDataChunks.get(currentChunkIndex).length))
      currentChunkIndex++;

    currentChunk = binaryDataChunks.get(currentChunkIndex);
    currentSize -= currentChunk.length;
    // the position referred to the target binary chunk
    int relativePosition = (int) (pos - currentSize);
    // the index of the first byte to be returned
    return relativePosition - 1;
  }</s>

<s>[{"type": "scopedVariable", "name": "currentSize"}, {"type": "methodInvocation", "to": ["binaryDataChunks"], "method": "get"}, {"type": "blocks", "blocks": [{"name": "while", "guard": "pos >= (currentSize = length)", "contents": []}]}, {"type": "methodInvocation", "to": ["binaryDataChunks"], "method": "get"}, {"type": "scopedVariable", "name": "relativePosition"}, "return"]</s>

<s>[{"type": "scopedVariable", "name": "cu